In [3]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is E6E0-67EF

 Directory of C:\Users\GuilhermeYambanisTho\Desktop\IME\CovidMDP\notebooks\spread_modeling

05/18/2020  10:29 PM    <DIR>          .
05/18/2020  10:29 PM    <DIR>          ..
05/18/2020  10:27 PM    <DIR>          .ipynb_checkpoints
05/13/2020  06:09 PM            22,179 Automated Planning Modeling.ipynb
05/18/2020  10:29 PM             4,406 Old Create Graph that Worked.ipynb
04/29/2020  08:15 PM            43,583 Recovery model.ipynb
05/15/2020  12:14 PM            90,133 Recovery Modeling.ipynb
04/27/2020  10:49 PM           229,044 Relaxed Caveman Spread.ipynb
05/13/2020  10:53 AM           141,842 sigmoid lambda.ipynb
05/13/2020  09:48 AM         3,598,276 Simple policy implementation.ipynb
04/26/2020  01:46 PM           105,282 spreadRelaxedCaveman.png
05/13/2020  10:54 AM           126,948 Testing Replicability.ipynb
05/15/2020  10:01 PM             1,991 time series plot test.ipynb
              10 File(s)      4

In [4]:
import pyreadstat
import pandas as pd
import numpy as np
import networkx as nx
import warnings
import itertools
from tqdm import tqdm


df, meta = pyreadstat.read_sav("..\\..\\data\\raw\\OD 2017\\Banco de dados\\OD_2017.sav")
df.drop_duplicates(subset=['zona', 'zona_esc', 'zonatra1', 'id_pess'], inplace=True)

df = df[df['muni_dom'] == 36]
max_zona = int(df.zona.max())

trab_esc = df[['zona', 'zona_esc', 'zonatra1', 'id_pess', 'id_dom', 'idade', 'renda_fa']].copy()
trab_esc.rename(columns={'zona':'home_zone',
                        'zona_esc':'school',
                        'zonatra1':'work',
                        'id_pess': 'id', 'id_dom': 'house_id'}, inplace=True)

trab_esc = trab_esc[~(trab_esc['work'] > max_zona) & 
        ~(trab_esc['school'] > max_zona) & ~(trab_esc['home_zone'] > max_zona)].reset_index(drop=True)

quantile = 1 - 5730524/12038175

def has_private_healthcare(family_income):
    return family_income >= 4000

trab_esc['private_healthcare'] = trab_esc['renda_fa'].apply(has_private_healthcare)

G = nx.Graph()

def add_person_to_graph(person):
    G.add_node(person['id'],
               work = person['work'],
               school = person['school'],
               home = person['home_zone'],
               age = person['idade'],
               private_healthcare = person['private_healthcare']
    )    
    
trab_esc.apply(add_person_to_graph, axis=1)
print('Adicionando Pessoas')
print(len(G.nodes))
print(len(G.edges))
print(50*'*')

print('Adicionando Vertices de Casas')

def add_edge(person1, person2, edge_type, edge_zone):
        G.add_edge(person1, person2, edge_type=edge_type, zone=edge_zone)
        
for h in tqdm(trab_esc['house_id'].unique()):
    tmp = trab_esc[trab_esc['house_id'] == h]
    zone = tmp['home_zone'].value_counts().index[0]
    if len(tmp) > 1:
        for p1, p2 in list(itertools.combinations(tmp['id'].values, 2)):
            add_edge(p1, p2, 'house', zone)


print(len(G.nodes))
print(len(G.edges))
print(50*'*')

for w in tqdm(trab_esc['work'].unique()):
    if pd.notna(w):
        tmp = trab_esc[trab_esc['work'] == w]
        zone = tmp['work'].value_counts().index[0]

        if len(tmp) > 1:
            possible_combinations = np.array(list(itertools.combinations(tmp['id'].values, 2)))
            size_comb = int(len(possible_combinations))
            index_combs = np.random.choice(size_comb, size=int(size_comb*.05))
            real_combs = possible_combinations[index_combs]        
            for p1, p2 in real_combs:
                add_edge(p1, p2, 'work', zone)
                
print(len(G.nodes))
print(len(G.edges))
print(50*'*')

for s in tqdm(trab_esc['school'].unique()):
    if pd.notna(s):
        tmp = trab_esc[trab_esc['school'] == s]
        zone = tmp['school'].value_counts().index[0]

        if len(tmp) > 1:
            possible_combinations = np.array(list(itertools.combinations(tmp['id'].values, 2)))
            size_comb = int(len(possible_combinations))
            index_combs = np.random.choice(size_comb, size=int(size_comb*.25))
            real_combs = possible_combinations[index_combs]        
            for p1, p2 in real_combs:
                add_edge(p1, p2, 'school', zone)
                
print(len(G.nodes))
print(len(G.edges))
print(50*'*')

nx.write_gpickle(G, 'Grafo_Zonas_SP.gpickle')

  0%|                                                                              | 16/21708 [00:00<02:16, 158.83it/s]

Adicionando Pessoas
55492
0
**************************************************
Adicionando Vertices de Casas


  1%|▋                                                                                 | 3/339 [00:00<00:14, 23.86it/s]

55492
61634
**************************************************


  3%|██▋                                                                              | 11/336 [00:00<00:03, 93.72it/s]

55492
155827
**************************************************


100%|███████████████████████████████████████████████████████████████████████████████| 336/336 [00:00<00:00, 386.89it/s]


55492
253611
**************************************************
